In [1]:
pip install -q datasets trl huggingface_hub[hf_transfer] python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [3]:
from datasets import features, load_dataset
from transformers import AutoModelForVision2Seq, AutoProcessor
import torch
from trl import DPOConfig, DPOTrainer
from peft import LoraConfig
from dotenv import load_dotenv
from huggingface_hub import login
import wandb

load_dotenv()  # This loads the variables from the .env file
wandb_api_key = os.getenv("WANDB_API_KEY")
hf_token = os.getenv("HF_TOKEN")

if not wandb_api_key or not hf_token:
    raise ValueError("WANDB_API_KEY or HF_TOKEN is not set in environment variables.")

wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ashetty21 (ashetty21-university-of-california-berkeley). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# Load the model and processor
model = AutoModelForVision2Seq.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.bfloat16, device_map="auto")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", do_image_splitting=False)


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [5]:
# Load the dataset
dataset = load_dataset("abshetty/llava-chains-10k-individual", split="train")

README.md:   0%|          | 0.00/800 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9314 [00:00<?, ? examples/s]

In [6]:
def format(example):
    # Prepare the input for the chat template
    prompt = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": example["question_full"]}]}]
    chosen = [{"role": "assistant", "content": [{"type": "text", "text": example["best_response"]}]}]
    rejected = [{"role": "assistant", "content": [{"type": "text", "text": example["worst_response"]}]}]
    # Apply the chat template
    prompt = processor.apply_chat_template(prompt, tokenize=False)
    chosen = processor.apply_chat_template(chosen, tokenize=False)
    rejected = processor.apply_chat_template(rejected, tokenize=False)
    # Resize the image to ensure it fits within the maximum allowable
    # size of the processor to prevent OOM errors.
    max_size = processor.image_processor.size["shortest_edge"] // 2
    example["image"].thumbnail((max_size, max_size))
    return {"images": [example["image"]], "prompt": prompt, "chosen": chosen, "rejected": rejected}

# Apply the formatting function to the dataset
dataset = dataset.map(format, remove_columns=dataset.column_names, num_proc=32)

# Make sure that the images are decoded, it prevents from storing bytes.
# More info here https://github.com/huggingface/blog/pull/2148#discussion_r1667400478
f = dataset.features
f["images"] = features.Sequence(features.Image(decode=True))
dataset = dataset.cast(f)


Map (num_proc=32):   0%|          | 0/9314 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9314 [00:00<?, ? examples/s]

In [7]:
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)

# Extract the train and eval sets
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [11]:
eval_dataset[224]

{'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=168x168>],
 'prompt': 'USER: <image>\nRespond to the following user query in a comprehensive and detailed way. But first write down your internal thoughts. This must include your draft response and its evaluation. Write your thoughts after “Here is my thought process:” and write your final response after “Here is my response:”"\n\nQuestion: What do the items in the soup look like?\n\nAvailable choices:\n1. apple pie\n2. snails\n3. doritos\n4. squid ',
 'chosen': 'ASSISTANT: Here is my thought process: The soup is a creamy-looking dish with chips on top, which could be a combination of cheese and chips. The chips are likely to be a type of snack food, such as Doritos. The soup could be a cream soup, which is a popular choice for chips. The presence of chips on top of the soup indicates that it might be a popular dish, often enjoyed with chips as a topping.\n\nHere is my response: The soup in the image consists of a cream

In [12]:
# Train the model
training_args = DPOConfig(
    output_dir="lora-12-07-individual",
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps=1,
    learning_rate=1e-5,
    beta=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    num_train_epochs=2,
 #   weight_decay=0.01,
 #   rpo_alpha=0.1,
    dataset_num_proc=32,  # tokenization will use 32 processes
    dataloader_num_workers=32,  # data loading will use 32 workers
    logging_steps=1,
)

# Define LoRA configuration with specified rank
lora_config = LoraConfig(
    r=64,  # Set rank to 64
    lora_alpha=128,  # Set scaling factor to 128
    target_modules="all-linear",  # Target all linear layers
    lora_dropout=0.1,
)

trainer = DPOTrainer(
    model,
    ref_model=None,  # not needed when using peft
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    peft_config=lora_config,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Extracting prompt from train dataset (num_proc=32):   0%|          | 0/8382 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=32):   0%|          | 0/8382 [00:00<?, ? examples/s]

Extracting prompt from eval dataset (num_proc=32):   0%|          | 0/932 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=32):   0%|          | 0/932 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=32):   0%|          | 0/8382 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=32):   0%|          | 0/932 [00:00<?, ? examples/s]

In [13]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,0.693100,0.693147,0.000000,0.000000,0.000000,0.000000,-1274.629883,-1265.640259,-2.623588,-2.621376
2,0.693100,0.693916,-0.037762,-0.036985,0.491416,-0.000776,-1275.007568,-1266.010254,-2.629286,-2.626525
3,0.694000,0.687393,-0.167899,-0.182494,0.557940,0.014595,-1276.308716,-1267.465210,-2.621834,-2.618717
4,0.681900,0.677466,-0.496860,-0.550424,0.559013,0.053564,-1279.598389,-1271.144531,-2.609805,-2.606745
5,0.680500,0.660895,-0.641326,-0.753155,0.581545,0.111829,-1281.043091,-1273.171997,-2.577698,-2.575343
6,0.654900,0.629462,-0.611703,-0.807862,0.634120,0.196159,-1280.746948,-1273.718872,-2.536690,-2.534820
7,0.632500,0.595528,0.066450,-0.220021,0.698498,0.286472,-1273.965210,-1267.840576,-2.490972,-2.487846
8,0.581800,0.597540,1.112764,0.676467,0.681330,0.436297,-1263.502197,-1258.875488,-2.427533,-2.423091
9,0.610900,0.593227,1.407010,0.817903,0.685622,0.589107,-1260.559692,-1257.461182,-2.366782,-2.361813
10,0.583600,0.558585,0.772473,0.038619,0.708154,0.733854,-1266.905029,-1265.254150,-2.326356,-2.322112


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=64, training_loss=0.5267020813189447, metrics={'train_runtime': 34296.293, 'train_samples_per_second': 0.489, 'train_steps_per_second': 0.002, 'total_flos': 0.0, 'train_loss': 0.5267020813189447, 'epoch': 1.954198473282443})

In [ ]:
# Saving the fine-tuned model and tokenizer
trainer.model.save_pretrained("final_checkpoint")
processor.save_pretrained("final_checkpoint")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


['final_checkpoint/processor_config.json']

In [14]:
trainer.push_to_hub("abshetty/llava-lora-individual")



/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


  0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

(…).tfevents.1733618263.47125edf3cf9.1695.0:   0%|          | 0.00/100k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.07k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abshetty/lora-12-07-individual/commit/909bdf8cd3fc4a9fe006beb999d3a54e91f7bbb4', commit_message='abshetty/llava-lora-individual', commit_description='', oid='909bdf8cd3fc4a9fe006beb999d3a54e91f7bbb4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abshetty/lora-12-07-individual', endpoint='https://huggingface.co', repo_type='model', repo_id='abshetty/lora-12-07-individual'), pr_revision=None, pr_num=None)